# Imports

Data paths to modify (where data files should be saved)

In [1]:
STOP_DATA_DIR = "/home/guillaume/Documents/OpenData/Switzerland_transport/BAV_Liste/"
TIME_DATA_DIR = "/home/guillaume/Documents/OpenData/Switzerland_transport/Actual_data/"

Plot path to modify (where figures are saved)

In [2]:
PLOT_DIR = "plots/real_data/"

Automatic re-import upon source changes

In [3]:
%load_ext autoreload
%autoreload 2

Imports

In [4]:
import itertools
import os

from ipywidgets import interact, interact_manual, fixed
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
from src import *

Plot config

In [6]:
%matplotlib inline

Absolute paths

# Downloading / reading the data

## Stops data

Source: https://opentransportdata.swiss/en/dataset/bav_liste

In [7]:
stops_path = STOP_DATA_DIR + "stops.parquet"

In [8]:
download_stops_data(os.path.join(STOP_DATA_DIR, "stops.xlsx"))

In [11]:
stops = read_stops_data(os.path.join(STOP_DATA_DIR, "stops.xlsx"))

Computing latitude :   0%|          | 0/28757 [00:00<?, ?it/s]

Computing longitude :   0%|          | 0/28757 [00:00<?, ?it/s]

In [14]:
stops.to_parquet(stops_path)

In [15]:
stops = pd.read_parquet(stops_path)

## Time data

Before executing the next few cells/go to https://drive.google.com/drive/folders/1SVa68nJJRL3qgRSPKcXY7KuPN9MuHVhJ and download each monthly zip archive into a folder named `zip` within directory `TIME_DATA_DIR`. In this same directory, create other folders named `parquet_daily` and `parquet_monthly`.

In [ ]:
MONTHLY_ZIP_DIR_PATH = os.path.join(TIME_DATA_DIR, "zip")
DAILY_PARQUET_DIR_PATH = os.path.join(TIME_DATA_DIR, "parquet_daily")
MONTHLY_PARQUET_DIR_PATH = os.path.join(TIME_DATA_DIR, "parquet_monthly")

Beware: the following cell can take several hours to run.

In [ ]:
unzip_months_store_days(MONTHLY_ZIP_DIR_PATH, DAILY_PARQUET_DIR_PATH)
read_days_store_months(DAILY_PARQUET_DIR_PATH, MONTHLY_PARQUET_DIR_PATH)

In [ ]:
data = read_months_return_full(MONTHLY_PARQUET_DIR_PATH, years=[2018, 2019])

# Preprocessing

In [ ]:
data.dtypes

Drop the columns we won't use

In [ ]:
data = drop_useless_columns(data)

Uncategorize categorical variables for future group-bys

In [ ]:
data = uncategorize(data)

Remove skipped or unplanned stops and cancelled trips

In [ ]:
data = remove_skipped_unplanned_cancelled(data)

Select the event types we want

In [ ]:
data = keep_only_arrivals(data)

# EDA and cleaning

In [ ]:
data.dtypes

## Data quantity

Size

In [ ]:
data.shape

In [ ]:
data.memory_usage() / 1e9

In [ ]:
data.memory_usage().sum() / 1e9

Month heterogeneity

In [ ]:
months_since_start = 12*(data["date"].dt.year-2018) + data["date"].dt.month
month_freq = data.groupby(months_since_start).size()

In [ ]:
plt.close()
plt.bar(month_freq.index, month_freq)
plt.xlabel("Months since Jan 2018")
plt.ylabel("Nb of observations")
plt.savefig(PLOT_DIR + "month_freq.pdf")
plt.show()

In [ ]:
normal_months = month_freq[
    month_freq > 0.75*month_freq.median()
].index.tolist()

is_normal_month = months_since_start.isin(normal_months)

In [ ]:
is_normal_month.mean()

Weekday heterogeneity

In [ ]:
weekday_freq = data.groupby(
    data["date"].dt.weekday
).size()

In [ ]:
plt.close()
plt.bar(weekday_freq.index + 1, weekday_freq)
plt.xlabel("Weekday")
plt.ylabel("Nb of observations")
plt.savefig(PLOT_DIR + "weekday_freq.pdf")
plt.show()

In [ ]:
is_weekday = data["date"].dt.weekday <= 4

In [ ]:
is_weekday.mean()

Hour heterogeneity

In [ ]:
hour_freq = data.groupby(
    data["event_time_planned"].dt.hour
).size()

In [ ]:
plt.close()
plt.bar(hour_freq.index + 1, hour_freq)
plt.xlabel("Hour of day")
plt.ylabel("Nb of observations")
plt.savefig(PLOT_DIR + "hour_freq.pdf")
plt.show()

In [ ]:
is_evening_peak = (
    (17 <= data["event_time_planned"].dt.hour)
    & (data["event_time_planned"].dt.hour <= 19)
)

In [ ]:
is_evening_peak.mean()

Restrict dataset to a coherent subset in terms of data quantity

In [ ]:
data = data[is_normal_month & is_weekday & is_evening_peak]

Is there still date heterogeneity?

In [ ]:
date_freq = data.groupby(
    data["date"].dt.date
).size()

In [ ]:
plt.close()
plt.scatter(date_freq.index, date_freq, alpha=0.5, marker=".")
plt.xlabel("Date")
plt.ylabel("Nb of observations")
plt.savefig(PLOT_DIR + "date_freq.pdf")
plt.show()

In [ ]:
normal_dates = date_freq[
    (date_freq > 0.75*date_freq.median())
    & (date_freq < 1.25*date_freq.median())
].index.tolist()

In [ ]:
is_normal_date = data["date"].isin(normal_dates)

In [ ]:
is_normal_date.mean()

In [ ]:
data = data[is_normal_date]

New data size

In [ ]:
data.shape

In [ ]:
data.memory_usage().sum() / 1e9

## Data quality

Add next trip event in each row to allow for edge analysis

In [ ]:
data = add_next_event(data)

Remove self-loops

In [ ]:
data = data[data["stop_id"].values != data["next_stop_id"].values]

Add edge duration and delay columns

In [ ]:
data = add_delay_columns(data)

Analyze edge durations

In [ ]:
for col in [
    "edge_duration_planned",
    "edge_duration_real",
    "event_delay",
    "edge_delay",
]:
    plt.close()
    plt.yscale("log")
    data[col].hist(bins=50, color="red")
    plt.ylabel("Occurrences")
    plt.xlabel(col.replace("_", " ").capitalize())
    plt.savefig(PLOT_DIR + "histogram_nocleaning_{}.pdf".format(col))
    plt.show()

Remove outliers

In [ ]:
filtered_data = remove_outliers_delays(
    data,
    min_edge_duration_planned=0,
    max_edge_duration_planned=10,
    min_edge_duration_real=-np.inf,
    max_edge_duration_real=np.inf,
    min_edge_delay=-2,
    max_edge_delay=10,
    min_event_delay=-10,
    max_event_delay=30,
)

In [ ]:
data.shape, filtered_data.shape

In [ ]:
for col in [
    "edge_duration_planned",
    "edge_duration_real",
    "event_delay",
    "edge_delay",
]:
    plt.close()
    plt.yscale("log")
    filtered_data[col].hist(color="green", bins=50)
    plt.ylabel("Occurrences")
    plt.xlabel(col.replace("_", " ").capitalize())
    plt.savefig(PLOT_DIR + "histogram_aftercleaning_{}.pdf".format(col))
    plt.show()

In [ ]:
data = filtered_data

## Network

In [ ]:
data.groupby("stop_id").size().shape

In [ ]:
data.groupby(["stop_id", "next_stop_id"]).size().shape

In [ ]:
G = build_network(data, stops, max_edge_rank=200)

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
plot_network(G, ax)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.axis("scaled")
plt.savefig(PLOT_DIR + "map_zurich.pdf")

Define edge ids

In [ ]:
edge_ids = pd.Series(
    index=pd.MultiIndex.from_tuples(list(G.edges)),
    data=np.arange(G.number_of_edges()),
    name="edge_id",
    dtype="Int64"
)

Add ids to graph and data

In [ ]:
for stop_id, next_stop_id in edge_ids.index:
    G.edges[stop_id, next_stop_id]["edge_id"] = edge_ids[(stop_id, next_stop_id)]

In [ ]:
data = data.join(edge_ids, on=("stop_id", "next_stop_id"))
data = data[data["edge_id"].notnull()]

In [ ]:
data.shape

## Centering

In [ ]:
edge_delay_center = data.groupby("edge_id")["edge_delay"].mean()
edge_delay_center.name = "edge_delay_center"

In [ ]:
data = data.join(edge_delay_center, on="edge_id")
data["centered_edge_delay"] = data["edge_delay"] - data["edge_delay_center"]

# Transition estimation

In [ ]:
map_network(G)

In [ ]:
%matplotlib notebook

In [ ]:
plt.close()
fig, ax = plt.subplots(figsize=(6, 7))
interact_manual(
    estimate_transition_and_plot,
    data=fixed(data),
    h0=fixed(0),
    freq=(1, 10, 1),
    log_lambda0=(-3, 0, 0.01),
    log_weight_threshold=(-2, 0, 0.1),
    G=fixed(G),
    ax=fixed(ax),
    continuous_update=False
)

In [ ]:
%matplotlib inline

In [ ]:
lambda0_range = np.logspace(-2, 0, 100)
freq_range = [1, 2, 3, 5, 7, 10, 15]

In [ ]:
dist = get_distance_matrix(G)

In [ ]:
mean_interac_dists_by_freq = {}
densities_by_freq = {}

for freq in freq_range:
    thetas = estimate_transition(
        data=data, freq=freq, lambda0_range=lambda0_range
    )
    mean_interac_dists_by_freq[freq] = np.array([
        np.sum(np.abs(theta) * dist) / np.sum(np.abs(theta)) for theta in thetas
    ])
    densities_by_freq[freq] = np.array([
        np.mean(np.abs(theta) > 1e-2) * theta.shape[0] for theta in thetas
    ])

In [ ]:
plt.close()
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8, 8))

ax[1].axhline(y=0, color="k", linestyle="dashed")
ax[0].axhline(y=0, color="k", linestyle="dashed")

ax[1].axhline(y=dist.mean(), color="k", label="mean inter-edge dist", linestyle="dotted")

for freq in freq_range:
    ax[0].plot(
        lambda0_range, densities_by_freq[freq],
        label="$\\Delta t = {}$ min".format(freq),
        marker=MARKERS[k]
    )
    ax[1].plot(
        lambda0_range, mean_interac_dists_by_freq[freq],
        label="$\\Delta t = {}$ min".format(freq),
        marker=MARKERS[k]
    )

ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[0].set_ylabel("Mean sparsity level $s$ of $\\theta$")
ax[0].legend()

ax[1].set_xscale("log")
ax[1].set_ylabel("Average interaction distance [min]")
ax[1].set_xlabel("Regularization parameter $\\lambda_0$")
ax[1].legend()

plt.tight_layout()
plt.savefig(PLOT_DIR + "regularization_effect.pdf")
plt.show()

In [ ]:
data["edge_duration_planned"].mean()